In [82]:
import telebot
from telebot import types
import pickle
import pandas as pd
import numpy as np

In [113]:
with open('token.txt') as f:
    token = f.readline()
bot = telebot.TeleBot(token)

with open('model.pkl', 'rb') as f:
    clf = pickle.load(f)

df = pd.read_csv('results.csv', index_col=0)

In [116]:
user_dict = {}


class Estate:
    def __init__(self, num):
        self.rooms = num
        self.square = None
        self.floor = None
        self.max_floor = None
        self.balcony = None
        self.bathroom = None
        self.year = None
        
@bot.message_handler(commands=['help', 'start'])
def send_welcome(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Предоставить данные о квартире")
    markup.add(btn1)
    msg = bot.reply_to(message, """\
Здравствуйте! Я - бот, который подскажет цену на вашу квартиру в г. Пермь и отправит ссылки на похожие варинты квартир.
Мне будут необходимы следующие данные:
- количество комнат
- метраж квартиры
- этаж
- этажность дома
- балкон или лоджия
- раздельный или совмещенный санузел
- год постройки дома.
""", reply_markup=markup)
    bot.register_next_step_handler(msg, first_step)


def first_step(message):
    msg = bot.reply_to(message, 'Сколько комнат в квартире?')
    bot.register_next_step_handler(msg, process_rooms_step)


def process_rooms_step(message):
    try:
        chat_id = message.chat.id
        rooms = message.text
        if not rooms.isdigit():
            msg = bot.reply_to(message, 'Количество комнат должно быть числом.\nСколько комнат в квартире?')
            bot.register_next_step_handler(msg, process_rooms_step)
            return
        estate = Estate(rooms)
        user_dict[chat_id] = estate
        msg = bot.reply_to(message, 'Какая площадь квартиры?')
        bot.register_next_step_handler(msg, process_square_step)
    except Exception as e:
        bot.reply_to(message, 'Упс!')


def process_square_step(message):
    try:
        chat_id = message.chat.id
        square = message.text
        if not square.isdigit():
            msg = bot.reply_to(message, 'Площадь должна быть числом. Какая площадь квартиры?')
            bot.register_next_step_handler(msg, process_square_step)
            return
        estate = user_dict[chat_id]
        estate.square = square
        msg = bot.reply_to(message, 'Какой этаж?')
        bot.register_next_step_handler(msg, process_max_floor_step)
    except Exception as e:
        bot.reply_to(message, 'Упс!')


def process_max_floor_step(message):
    try:
        chat_id = message.chat.id
        floor = message.text
        if not floor.isdigit():
            msg = bot.reply_to(message, 'Этаж должен быть числом. Какой этаж?')
            bot.register_next_step_handler(msg, process_max_floor_step)
            return
        estate = user_dict[chat_id]
        estate.floor = floor
        msg = bot.reply_to(message, 'Скольки этажный дом?')
        bot.register_next_step_handler(msg, process_balcony_step)
    except Exception as e:
        bot.reply_to(message, 'Упс!')
        
def process_balcony_step(message):
    try:
        chat_id = message.chat.id
        max_floor = message.text
        if not max_floor.isdigit():
            msg = bot.reply_to(message, 'Этажность должна быть числом. Сколько этажей дом?')
            bot.register_next_step_handler(msg, process_balcony_step)
            return
        estate = user_dict[chat_id]
        estate.max_floor = max_floor
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('балкон', 'лоджия')
        msg = bot.reply_to(message, 'Какая терраса?', reply_markup=markup)
        bot.register_next_step_handler(msg, process_bathroom_step)
    except Exception as e:
        bot.reply_to(message, 'Упс!')


def process_bathroom_step(message):
    try:
        chat_id = message.chat.id
        balcony = message.text
        estate = user_dict[chat_id]
        if (balcony == u'балкон') or (balcony == u'лоджия'):
            estate.balcony = balcony
        else:
            raise Exception()
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('раздельный', 'совмещенный')
        msg = bot.reply_to(message, 'Санузел раздельный или совмещенный?', reply_markup=markup)
        bot.register_next_step_handler(msg, process_year_step)
    except Exception as e:
        bot.reply_to(message, 'Упс!')


def process_year_step(message):
    try:
        chat_id = message.chat.id
        bathroom = message.text
        estate = user_dict[chat_id]
        if (bathroom == u'раздельный') or (bathroom == u'совмещенный'):
            estate.bathroom = bathroom
        else:
            raise Exception()
        msg = bot.reply_to(message, 'Какого года дом?')
        bot.register_next_step_handler(msg, process_ending_step)
    except Exception as e:
        bot.reply_to(message, 'Упс!')


def process_ending_step(message):
    try:
        chat_id = message.chat.id
        year = message.text
        if not year.isdigit():
            msg = bot.reply_to(message, 'Этаж должен быть числом. Какой этаж?')
            bot.register_next_step_handler(msg, process_ending_step)
            return
        estate = user_dict[chat_id]
        estate.year = year
        markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
        markup.add('Узнать цену')
        msg = bot.reply_to(message, f'''Ваши данные по квартире:
- количество комнат: {estate.rooms}
- метраж квартиры: {estate.square}
- этаж: {estate.floor}
- этажность дома: {estate.max_floor}
- балкон или лоджия: {estate.balcony}
- раздельный или совмещенный санузел: {estate.bathroom}
- год постройки дома: {estate.year}''', reply_markup=markup)
        bot.register_next_step_handler(msg, predict_price)
    except Exception as e:
        bot.reply_to(message, 'Упс!')


def predict_price(message):
    try:
        chat_id = message.chat.id
        estate = user_dict[chat_id]
        
        X_pred = pd.DataFrame({'Rooms': [int(estate.rooms)], 
                       'Area': [float(estate.square)], 
                       'Floor': [int(estate.floor)], 
                       'Max_house_floor': [int(estate.max_floor)],
                       'Year_built': [int(estate.year)],
                       'balcony_or_loggia_балкон': [0],
                       'balcony_or_loggia_лоджия': [0],
                       'Bathroom_раздельный': [0],
                       'Bathroom_совмещенный': [0],
                       'low-floor': [0],
                       'middle-floor': [0],
                       'higher_middle-floor': [0],
                       'multi-floor': [0],
                       'higher_multi-floor': [0],
                       })
        X_pred = pd.get_dummies(X_pred)
        if estate.bathroom == 'разделенный':
            X_pred['Bathroom_раздельный'] = 1
        else:
            X_pred['Bathroom_совмещенный'] = 1
        if estate.balcony == 'балкон':
            X_pred['balcony_or_loggia_балкон'] = 1
        else:
            X_pred['balcony_or_loggia_лоджия'] = 1
        X_pred['low-floor'] = np.where(X_pred['Max_house_floor'] < 4, 1, 0)
        X_pred['middle-floor'] = np.where((X_pred['Max_house_floor'] > 3) & (X_pred['Max_house_floor'] < 6), 1, 0)
        X_pred['higher_middle-floor'] = np.where((X_pred['Max_house_floor'] > 6) & (X_pred['Max_house_floor'] < 11), 1, 0)
        X_pred['multi-floor'] = np.where((X_pred['Max_house_floor'] > 11) & (X_pred['Max_house_floor'] < 25), 1, 0)
        X_pred['higher_multi-floor'] = np.where(X_pred['Max_house_floor'] > 25, 1, 0)
        price = int(clf.predict(X_pred)[0])
        price -= price % -10000
        price = str(price)
        if len(price) == 7:
            price = price[0] + ' ' + price[1:4] + ' ' + price[4:]
        elif len(price) == 8:
            price = price[:2] + ' ' + price[1:4] + ' ' + price[4:]
        
        neighbors = clf.kneighbors(X_pred, return_distance=False)[0]
        links = []
        for i in neighbors:
            links.append(df.iloc[i, -1])
            
        bot.send_message(chat_id, f'''Ориентировочная стоимость вашей квартиры: <b>{price}</b> руб.
Похожие объявления:
1 - {links[0]}
2 - {links[1]}
3 - {links[2]}
4 - {links[3]}''', parse_mode='html')
    except Exception as e:
        bot.reply_to(message, 'Упс!')

In [117]:
bot.polling(none_stop=True)